In [2]:
import pandas as pd
import numpy as np
import torch
import datetime
from methods import *

In [2]:
data = pd.read_csv("voxmeter.csv")
data

,party_letter,votes,date
0,A,18.6,2025-12-14
1,B,5.2,2025-12-14
2,C,7.8,2025-12-14
3,F,14.3,2025-12-14
4,H,1.7,2025-12-14
5,I,11.7,2025-12-14
6,M,2.4,2025-12-14
7,O,8.0,2025-12-14
8,V,11.7,2025-12-14
9,Æ,9.2,2025-12-14


In [5]:
party_ = party(data, "A")
party_.party_data

,party_letter,votes,date
0,A,18.6,2025-12-14
12,A,18.0,2025-12-07
24,A,19.4,2025-11-30
36,A,19.3,2025-11-23
48,A,19.9,2025-11-16
